# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
import os, glob
from macrohet import dataio, tile, visualise, notify
import numpy as np
from macrohet import visualise
import os
import re
import numpy as np
import cv2
import btrack
import zarr

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [136]:
%%time
expt_ID = 'PS0000'

base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 11.8 s, sys: 100 ms, total: 11.9 s
Wall time: 11.9 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [137]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=False)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
4   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
5   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99
6   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99

In [133]:
base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}'
row = acq_ID[0]
column = acq_ID[1]
image_dir = os.path.join(base_dir, 'acquisition/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # set_plane='max_proj',
                             )

In [135]:
images

dask.array<reshape, shape=(154, 2, 3, 6048, 6048), dtype=uint16, chunksize=(1, 2, 3, 2160, 2160), chunktype=numpy.ndarray>

### Load using Zarr

In [64]:
acq_ID = (3, 5)

In [65]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')

zarr_group = zarr.open(image_dir, mode='r')

In [66]:
%%time
images = zarr_group.images[:,:,:,...]
images.shape

CPU times: user 21.8 s, sys: 30.2 s, total: 51.9 s
Wall time: 2min 50s


(75, 2, 3, 6048, 6048)

In [6]:
images_max_proj = np.max(zarr_group.images, axis = 2)

In [67]:
images_max_proj = np.max(images, axis = 2)

In [19]:
images_max_proj.shape

(154, 2, 6048, 6048)

In [68]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}, full image stack')

viewer.add_image(images_max_proj, #images,
                 channel_axis = 1, 
                 colormap=['green', 'magenta'],
                 blending = 'additive', 
                 contrast_limits=[[0, 2400], [480, 1000]]
                )

[<Image layer 'Image' at 0x7fe5590f4dc0>,
 <Image layer 'Image [1]' at 0x7fec333e5f10>]

In [69]:
print()

In [122]:
import dask.array as da

In [123]:
da.asarray(zarr_group.images)

dask.array<array, shape=(75, 2, 3, 6048, 6048), dtype=uint16, chunksize=(1, 1, 1, 6048, 6048), chunktype=numpy.ndarray>

# Saving out

In [33]:
def remove_black_border(image):
    # Create a mask for non-black pixels (considering the alpha channel as well)
    non_black_mask = np.any(image[:, :, :3] != 0, axis=-1) & (image[:, :, 3] != 0)
    
    # Identify the rows and columns that contain non-black pixels
    non_black_rows = np.where(np.any(non_black_mask, axis=1))[0]
    non_black_cols = np.where(np.any(non_black_mask, axis=0))[0]
    
    if non_black_rows.size and non_black_cols.size:
        # Crop the image to the identified rows and columns
        cropped_image = image[non_black_rows.min():non_black_rows.max() + 1, 
                              non_black_cols.min():non_black_cols.max() + 1]
    else:
        # If no non-black pixels are found, return the original image
        cropped_image = image
    
    return cropped_image

from skimage import io

In [186]:
viewer.reset_view()

In [217]:
screenshot = viewer.screenshot()

In [210]:
screenshot.shape

(1079, 1465, 4)

In [218]:
cropped_screenshot = screenshot #remove_black_border(screenshot)

In [203]:
screenshot.shape

(1079, 1465, 4)

In [190]:
viewer.add_image(cropped_screenshot)

<Image layer 'cropped_screenshot [1]' at 0x7fe55a1688e0>

In [113]:
io.imsave(f'/mnt/SYNO/macrohet_syno/illustration/day24/f1/{expt_ID}.{acq_ID[0]}.{acq_ID[1]}.t0.single_tile.rgba.png', arr=cropped_screenshot, )

In [219]:
io.imsave(f'/mnt/SYNO/macrohet_syno/illustration/day24/f1/{expt_ID}.{acq_ID[0]}.{acq_ID[1]}.t-1.true_track_proj_IDs.rgba.png', arr=cropped_screenshot, )

In [87]:
io.imsave(f'/mnt/SYNO/macrohet_syno/illustration/day24/f1/{expt_ID}.{acq_ID[0]}.{acq_ID[1]}.t0.segmented_opaque.rgba.png', arr=cropped_screenshot, )

In [ ]:
'/mnt/SYNO/macrohet_syno/data/PS0000/labels/macrohet_seg_model/'

In [183]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/macrohet_seg_model/(3, 5).h5'), 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                # segmentation = reader.segmentation
                tracks = reader.tracks

[INFO][2024/07/16 06:08:14 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/data/PS0000/labels/macrohet_seg_model/(3, 5).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/PS0000/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2024/07/16 06:08:14 pm] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2024/07/16 06:08:14 pm] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2024/07/16 06:08:14 pm] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2024/07/16 06:08:15 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/data/PS0000/labels/macrohet_seg_model/(3, 5).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/data/PS0000/labels/macrohet_seg_model/(3, 5).h5


In [72]:
viewer.add_labels(segmentation)

<Labels layer 'segmentation' at 0x7fe557fbd670>

In [73]:
import btrack

In [184]:
tracks, _, _ = btrack.utils.tracks_to_napari([t for t in tracks if len(t) >= 34])

In [198]:
viewer.add_tracks(tracks, scale = (50, 5.04, 5.04, ))

<Tracks layer 'tracks [3]' at 0x7fe557f397f0>

In [194]:
images_max_proj[-1].shape

(2, 6048, 6048)

In [195]:
viewer.add_image(images_max_proj[-1], #images,
                 channel_axis = 0, 
                 colormap=['green', 'magenta'],
                 blending = 'additive', 
                 contrast_limits=[[0, 2400], [480, 1000]]
                )

[<Image layer 'Image [2]' at 0x7fec225419d0>,
 <Image layer 'Image [3]' at 0x7fec22548d00>]

INFO:OpenGL.acceleratesupport:No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


## scale

In [116]:
image_scale_m_per_pixel = 1.4949402023919043e-07
image_scale_um_per_pixel = image_scale_m_per_pixel*1E6

In [139]:
image_scale_um_per_pixel

0.14949402023919042

In [166]:
image_width = image_scale_um_per_pixel * 6048

In [169]:
40 * 200/(image_scale_um_per_pixel * 6048)

8.84818884818885

In [164]:
50/(image_scale_um_per_pixel * 2016)

0.16590354090354092

In [165]:
1/6

0.16666666666666666

In [162]:
image_width

301.3799448022079

In [159]:
actual_width = 40

In [161]:
image_width / actual_width

7.5344986200551975

In [148]:
Nmum = 50

In [155]:
px_per_Nmum = Nmum/image_scale_um_per_pixel


In [156]:
px_per_mm = 40/2016

In [157]:
length = px_per_Nmum * px_per_mm

In [158]:
length

6.6361416361416365

# save out as individual tile

In [97]:
viewer.add_image(images_max_proj[:,:,0:2016, 0:2016], 
                 channel_axis = 1, 
                 colormap=['green', 'magenta'],
                 blending = 'additive', 
                 contrast_limits=[[0, 2400], [480, 1000]]
                )

[<Image layer 'Image [2]' at 0x7fe557f81dc0>,
 <Image layer 'Image [3]' at 0x7fe557fcdd60>]

In [100]:
v = napari.Viewer(title = 'single tile')
v.add_image(images_max_proj[:,:,0:2016, 0:2016], 
            channel_axis = 1, 
            colormap=['green', 'magenta'],
            blending = 'additive', 
            contrast_limits=[[0, 2400], [480, 1000]]
            )

[<Image layer 'Image' at 0x7fe5580c08e0>,
 <Image layer 'Image [1]' at 0x7fec2ac3ef40>]